In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from stl import mesh
from mpl_toolkits.mplot3d import Axes3D
import pyvista as pv
import trimesh
import shapely.geometry
import shapely as sp
from shapely.geometry import LineString, Polygon
from scipy.interpolate import interp1d

def openImage(image_path, show_image=False, show_info=False):
    """
    Load an image file and convert it to grayscale for depth and CMYK for color.

    Args:
        image_path (str): Path to any readable image file (e.g., jpg, png, etc.).
        show_image (bool): If True, displays the CMYK channels as grayscale images.
        show_info (bool): If True, prints image details such as name, format, size, and colorspace.

    Returns:
        cyan (PIL.Image.Image): Cyan channel image (intensity values: 0-255).
        magenta (PIL.Image.Image): Magenta channel image (intensity values: 0-255).
        yellow (PIL.Image.Image): Yellow channel image (intensity values: 0-255).
        black (PIL.Image.Image): Key (black) channel image (intensity values: 0-255).
        grayscale_image (PIL.Image.Image): Grayscale image for depth mapping (intensity values: 0-255).
    """
    
    image = Image.open(image_path)
    grayscale_image = image.convert("L")

    if show_info:
        print(f"Image Name: {image_path}")
        print(f"Image Format: {image.format}")
        print(f"Image Size: {image.size} [Width x Height]")
        print(f"Image Mode: {image.mode}")
    
    if image.mode != "CMYK":
        cmyk_image = image.convert("CMYK")
    else:
        cmyk_image = image
    cyan, magenta, yellow, black = cmyk_image.split()
    colors = ["C","M","Y","K"]
    if show_image:
        fig, axs = plt.subplots(1, 4)
        for i, color in enumerate(cmyk_image.split()):
            axs[i].imshow(color, cmap="gray")
            axs[i].axis('off')
            axs[i].set_title(colors[i])
        plt.show()

    return cyan, magenta, yellow, black, grayscale_image

def createBookMesh(width, spine_thickness, depth, thickness, height, lip_dist=5, lip_thickness=2, lip_depth=2, lip_trim=5, show_profile=False):
    """
    Create a 2D profile for a book using lines and an elliptical spine, 
    then extrude it into a 3D solid.

    Args:
        width (float): Width of the book covers (mm).
        spine_thickness (float): Thickness of the spine (minor axis of the ellipse) (mm).
        depth (float): Depth of the spine (major axis of the ellipse) (mm).
        thickness (float): Thickness of the book covers (mm).
        height (float): Height of the book (extrusion length along Z-axis) (mm).
        show_profile (bool): If True, displays the 2D profile of the book.

    Returns:
        trimesh.Trimesh: The 3D extruded book mesh.
    """
    
    back_cover = [(0, 0), (width - spine_thickness, 0)]

    t = np.linspace(0, np.pi, 100)
    spine = [
        (
            width - spine_thickness + (spine_thickness / 2) * np.sin(theta),
            -depth / 2-(depth / 2) * np.cos(theta),
        )
        for theta in t
    ]

    front_cover = [(width - spine_thickness, -depth), (0, -depth)]
    points = back_cover + np.array(spine)[::-1].tolist() + front_cover
    profile = LineString(points)

    if show_profile:
        x, y = profile.xy
        plt.plot(x,y, label="Book Profile")
        plt.title("2D Book Profile")
        plt.xlabel("Width (mm)")
        plt.ylabel("Depth (mm)")
        plt.axis("equal")
        plt.legend()
        plt.grid(True)
        plt.show()
    
    outer_line = LineString(points)
    offset_line = sp.offset_curve(outer_line, -thickness)
    open_profile = LineString(list(outer_line.coords) + list(offset_line.coords)[::-1])
    open_polygon = Polygon(open_profile)

    lip_offset = sp.offset_curve(offset_line, -lip_depth)

    inner_coords = np.array(lip_offset.coords)
    outer_coords = np.array(offset_line.coords)
    inner_coords[inner_coords[:, 0] == 0, 0] = lip_trim
    outer_coords[outer_coords[:, 0] == 0, 0] = lip_trim
    trimmed_profile_coords = np.vstack([outer_coords, inner_coords[::-1]])
    
    lip_profile = LineString(trimmed_profile_coords)
    lip_polygon = Polygon(lip_profile)
    lip_mesh = trimesh.creation.extrude_polygon(lip_polygon, lip_thickness)
    lip_mesh.apply_translation([0, 0, lip_dist])
    lip_mesh_top = lip_mesh.copy()
    lip_mesh_top.apply_translation([0, 0, height - 2*lip_dist])
    
    book_mesh = trimesh.creation.extrude_polygon(open_polygon, height)

    mesh = trimesh.util.concatenate([book_mesh, lip_mesh, lip_mesh_top])
    
    return outer_line, mesh

def createBase(grayscale, spine_depth, depth, thickness, book_height, lip_params=None, show_info=True):
    """
    Generate a base 3D book mesh using an image and specified dimensions.

    This function processes a given image to extract grayscale and CMYK channels, calculates
    the aspect ratio of the image to determine the dimensions of the book mesh, and constructs
    a 3D mesh representing a book with front/back covers, a spine, and optional lip features.

    Args:
        image_path (str): Path to the input image file (e.g., JPEG or PNG).
        spine_depth (float): Depth of the spine (in mm).
        depth (float): Total depth of the book (in mm, including spine and covers).
        thickness (float): Thickness of the book covers (in mm).
        book_height (float): Height of the book (in mm, corresponds to image height).
        lip_params (dict, optional): Parameters for adding lips to the book covers:
            - "lip_dist" (float): Distance from the book edges to the lip (default: 5 mm).
            - "lip_thickness" (float): Thickness of the lip (default: 2 mm).
            - "lip_depth" (float): Depth of the lip (default: 2 mm).
            - "lip_trim" (float): Trim distance for the lip profile (default: 5 mm).
        show_info (bool): If True, prints the calculated dimensions of the book.

    Returns:
        book_profile (shapely.geometry.LineString): 2D profile of the book shape.
        book_mesh (trimesh.Trimesh): 3D mesh representation of the book.
        cmyk (list of PIL.Image.Image): List of the CMYK image channels (cyan, magenta, yellow, black).
    """
    
    image_width, image_height = grayscale.size
    

    aspect_ratio = image_width / image_height
    flat_book_width = book_height * aspect_ratio
    cover_width = (flat_book_width - depth) / 2

    if show_info:
        print(f"Flat book width: {flat_book_width:.2f} mm")
        print(f"Front/Back cover width: {cover_width:.2f} mm")
        print(f"Spine width: {spine_depth:.2f} mm")

    book_profile, book_mesh = createBookMesh(
        width=flat_book_width,
        spine_thickness=spine_depth,
        depth=depth,
        thickness=thickness,
        height=book_height,
        lip_dist=lip_params.get("lip_dist", 5) if lip_params else 0,
        lip_thickness=lip_params.get("lip_thickness", 2) if lip_params else 0,
        lip_depth=lip_params.get("lip_depth", 2) if lip_params else 0,
        lip_trim=lip_params.get("lip_trim", 5) if lip_params else 0,
        show_profile=False,
    )

    return book_profile, book_mesh, cmyk

def resampleCurve(points, n):
    """
    Resample a non-linear curve to generate `n` evenly spaced points.

    Args:
        points (list of tuple): List of (x, y) coordinates representing the curve.
        n (int): Number of evenly spaced points to generate.

    Returns:
        numpy.ndarray: Array of shape (n, 2) containing the new (x, y) coordinates.
    """
    points = np.array(points)
    x, y = points[:, 0], points[:, 1]
    
    distances = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    
    cumulative_length = np.insert(np.cumsum(distances), 0, 0)
    total_length = cumulative_length[-1]
    
    even_lengths = np.linspace(0, total_length, n)
    
    interp_x = interp1d(cumulative_length, x, kind='linear')
    interp_y = interp1d(cumulative_length, y, kind='linear')
    
    new_x = interp_x(even_lengths)
    new_y = interp_y(even_lengths)
    
    resampled_points = np.column_stack((new_x, new_y))
    
    return resampled_points


image_path = "way of kings.jpg" 
spine_depth = 20 
depth=150
thickness = 2  
book_height = 300 
lip_params = {
    "lip_dist": 3,
    "lip_thickness": 2,
    "lip_depth": 2,
    "lip_trim": 10,
}
cyan, magenta, yellow, black, grayscale = openImage(image_path, show_image=False, show_info=True)
cmyk = [cyan, magenta, yellow, black]

book_profile, book_mesh, cmyk = createBase(
    grayscale=grayscale,
    spine_depth=spine_depth,
    depth=depth,
    thickness=thickness,
    book_height=book_height,
    lip_params=lip_params,
)

Image Name: way of kings.jpg
Image Format: JPEG
Image Size: (1090, 720) [Width x Height]
Image Mode: RGB
Flat book width: 454.17 mm
Front/Back cover width: 152.08 mm
Spine width: 20.00 mm


In [2]:
def generateLithophane(grayscale_image, max_thickness=2.0, min_thickness=0.8):
    """
    Generate a 3D lithophane mesh from a grayscale image, including thickness.

    This function takes a grayscale image and generates a 3D mesh representation 
    of a lithophane. The height of each point is mapped from the pixel intensity, 
    where darker areas correspond to thicker sections and brighter areas correspond 
    to thinner sections. The function also constructs a back surface and side walls 
    to create a complete, closed 3D model suitable for 3D printing.

    Args:
        grayscale_image (PIL.Image.Image): Grayscale image input. The pixel intensities 
                                           are used to determine the depth of the lithophane.
        max_thickness (float): Maximum thickness of the lithophane (corresponding to dark areas).
        min_thickness (float): Minimum thickness of the lithophane (corresponding to bright areas).

    Returns:
        trimesh.Trimesh: A 3D lithophane mesh including the front surface, back surface, 
                         and side walls as a watertight model.
    """
    grayscale_array = np.array(grayscale_image)[::-1, :]
    height, width = grayscale_array.shape

    normalized = np.clip(grayscale_array / 255.0, 0.0, 1.0)

    z = min_thickness + (max_thickness - min_thickness) * (1 - normalized)

    x = np.linspace(0, width - 1, width)
    y = np.linspace(0, height - 1, height)
    x_grid, y_grid = np.meshgrid(x, y)

    front_vertices = np.column_stack((x_grid.ravel(), y_grid.ravel(), z.ravel()))
    
    back_vertices = np.column_stack((x_grid.ravel(), y_grid.ravel(), np.full_like(z.ravel(),0)))

    vertices = np.vstack((front_vertices, back_vertices))

    rows, cols = z.shape
    vertex_count = rows * cols
    faces = []

    for i in range(rows - 1):
        for j in range(cols - 1):
            v1 = i * cols + j
            v2 = v1 + 1
            v3 = v1 + cols
            v4 = v3 + 1

            faces.append([v1, v2, v3])
            faces.append([v2, v4, v3])

            b1, b2, b3, b4 = v1 + vertex_count, v2 + vertex_count, v3 + vertex_count, v4 + vertex_count
            faces.append([b1, b3, b2])
            faces.append([b2, b3, b4])

    for i in range(rows - 1):
        for j in range(cols - 1):
            v1 = i * cols + j
            v2 = v1 + 1
            v3 = v1 + cols
            v4 = v3 + 1

            b1, b2, b3, b4 = v1 + vertex_count, v2 + vertex_count, v3 + vertex_count, v4 + vertex_count

            faces.extend([
                [v1, b1, v2], [v2, b1, b2], 
                [v2, b2, v4], [v4, b2, b4],
                [v4, b4, v3], [v3, b4, b3], 
                [v3, b3, v1], [v1, b3, b1], 
            ])

    faces = np.array(faces)

    lithophane_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

    return lithophane_mesh

lithophane_mesh = generateLithophane(grayscale)

In [ ]:
vertices = deformVertices(grayscale, points)
x, y, z = zip(*vertices)
plt.scatter(x, y, s=1, label='(x, y)')
plt.show()